## Understanding the problem

We have a time dynamical system whose state is described by two variables, $x$ and $v$.

The question asks how we can approximate these time dynamical systems using a regression tree when the data is in the form
$$
((x_k, v_k), (x_{k+1}, v_{k+1}))
$$

We could use a single tree to make predictions for the next state, but we would get more accurate results if we used multiple regression trees each with a limited height?

## Code for task 4

In [1]:
from task1 import *

# generating data
T = np.arange(0, 20)

# initial state
x_k = np.array([0.5, 1.5])

# creating an array of data
# shape: (number of time steps (rows), number of entries in each row, dimension of each column (previous state and next state))
X = np.zeros((T.size, 2, 2))

for t in T:
    X[t, 0, :] = x_k
    # creating next state
    X[t, 1, 0] = 0.9*x_k[0] - 0.2*x_k[1]
    X[t, 1, 1] = 0.2*x_k[0] + 0.9*x_k[1]
    x_k = X[t, 1, :]

y = X[:, 1, 0]
X = X[:, 0]

In [2]:
print(y[0:2])
np.mean(y[0:2], axis=0)

[ 0.15  -0.155]


-0.002500000000000044

In [3]:
model = RegressionTree()
model.fit(X, y)
model.decision_path([0.5, 1.5])
print(y[0])

Length of queue: 1
x[0] >= -0.31973391403185003
x[0] >= -0.15500000000000005
x[0] >= 0.14999999999999997
x[0] >= 0.14999999999999997
x == 0.19788404900569195
0.14999999999999997


/home/xavier/miniconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3787: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/xavier/miniconda3/lib/python3.12/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/home/xavier/miniconda3/lib/python3.12/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/xavier/miniconda3/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/xavier/miniconda3/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
